# Imports & Installation

## Pip

In [ ]:
%%capture
!pip install macest
!pip install sdv
!pip install pyod
import os
os.kill(os.getpid(), 9)

In [ ]:
%%capture
# Installations
# !pip install numpy --upgrade
!pip install macest
# !pip install seaborn
# !pip install scipy --upgrade
# !pip install scikit-learn --upgrade
!pip install sdv
!pip install pyod

import os
os.kill(os.getpid(), 9)

In [ ]:
##Pips for CQR
# !pip install -U scikit-learn==0.21.3
# !pip install scikit-garden

# from skgarden import RandomForestQuantileRegressor


# Automatic pipeline

In [74]:
cd /content/drive/MyDrive/MLops/MLops_G2JN

/content/drive/MyDrive/MLops/MLops_G2JN


In [ ]:
!python main.py

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib

--------------------------------
--------------------------------
--------------------------------

Initializing Pipeline on Boston-Housing dataset...

Initial RMSE: 3.479232 

    count_samples  start_bin_value  end_bin_value  suspected_low_in_data_bins uncertainty_type
7              63        20.000000      22.142857                       False              NaN
8              62        22.142857      24.285714                       False              NaN
6              59        17.857143      20.000000                       False              NaN
4              37        13.571429      15.714286                       False              NaN
5              32        15.714286      17.857143                       False        Aleatoric
3              28        11.428571      13.571429   

## Imports

In [27]:
%%capture

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
import numpy as np  
import pandas as pd
from scipy import stats
import torch

import xgboost as xgb
import time
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from macest.regression import models as reg_mod
from sdv.tabular import TVAE #data generation
from pyod.models.knn import KNN #outliers removal
from macest.model_selection import KFoldConfidenceSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

pd.options.mode.chained_assignment = None  # default='warn'

In [26]:
# !pip freeze > newer_requirementsֹ.txt

## Drive 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
cd /content/drive/MyDrive/MLops

/content/drive/MyDrive/MLops


# Boston DataSet

## Load data

In [29]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_dataset = pd.read_csv("data/housing.csv", header=None, delimiter=r"\s+", names=column_names)
boston_dataset.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [30]:
X_bos, y_bos = boston_dataset.iloc[:,:-1], boston_dataset.iloc[:,-1]

#Motors DataSet

## Load data

In [9]:
# motors_dataset = pd.read_csv('data/freMTPL2freq.csv').sample(frac=0.1,random_state=1).reset_index(drop=True)
motors_dataset = pd.read_csv('data/freMTPL2freq.csv')
motors_dataset.head(5)

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,3.0,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,5.0,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,10.0,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,11.0,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


Convert Categorical features to Numerical Features

In [10]:
for col in ['Area','VehBrand','VehGas','Region']:
  d = {}
  for i, val in enumerate(motors_dataset[col].unique()):
    d[val] = i + 1
  motors_dataset[col] = motors_dataset[col].apply(lambda x: d[x])

In [11]:
original_motors_dataset = motors_dataset.copy()
# Calculate Frequency
motors_dataset['Frequency'] = motors_dataset['ClaimNb'] / motors_dataset['Exposure']

motors_dataset.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,Frequency
0,1.0,1,0.10,1,5,0,55,50,1,1,1217,1,10.000000
1,3.0,1,0.77,1,5,0,55,50,1,1,1217,1,1.298701
2,5.0,1,0.75,2,6,2,52,50,1,2,54,2,1.333333
3,10.0,1,0.09,2,7,0,46,50,1,2,76,3,11.111111
4,11.0,1,0.84,2,7,0,46,50,1,2,76,3,1.190476


In [12]:
# X_mot, y_mot = motors_dataset.drop(['ClaimNb'],axis=1), motors_dataset.iloc[:,1]
X_mot, y_mot = motors_dataset.drop(['IDpol', 'ClaimNb', 'Exposure', 'Frequency'],axis=1), motors_dataset['Frequency']


# Helper functions

##Assign bins

### OLD ASSIGN

In [ ]:
def joel_assign_bins(
    X,
    y,
    samples_per_bin=20,
    max_bins=100,
    method="mean",
    threshold=0.5,
):

    """
    Function which divides the data into bins
    Args:
        X, y: the data.
        samples_per_bin:
        max_bins:
        method:
        threshold:
    Return:
       bins_df:
       X:
    """

    if X.shape[0] <= 1000:
      samples_per_bin = X.shape[0] * 5 // 100 # up to 5% of samples per bin
    elif X.shape[0] <= 100000:
      samples_per_bin = X.shape[0] // 100 # up to 1% of samples per bin
    elif X.shape[0] <= 1000000:
      samples_per_bin = X.shape[0] // 1000 # up to 0.1% of samples per bin
    else :
      samples_per_bin = X.shape[0] // 10000 # up to 0.01% of samples per bin

    qty_bins = X.shape[0] // samples_per_bin
    print('samples per bin', samples_per_bin)

    num_of_bins2 = np.arange(0, X.shape[0], samples_per_bin)
    num_of_bins = []
    y.sort_values(inplace=True)
    for i in range(0, X.shape[0], samples_per_bin):
      num_of_bins.append( y.iloc[i] )

    #if num_of_bins > max_bins:
    #    num_of_bins = max_bins
    # num_of_bins = (y.max()-y.min())/ 
    # num_of_bins = 10


    bin_assignment, bin_ranges = pd.cut(y, num_of_bins2, labels=False, retbins=True)
    # bin_assignment, bin_ranges = pd.cut(y, num_of_bins, labels=False, retbins=True)
    print('bin_assignment',bin_assignment)
    X["bins"] = bin_assignment
    print('x',X)
    bin_vals = bin_assignment.value_counts().sort_index()
    print('binvals',bin_vals)
    bin_vals = bin_vals.reindex(range(num_of_bins), fill_value=0)
    bin_ranges[0] = 0
    bin_ranges[-1] = bin_ranges[-1] * 10
    bins_df = pd.DataFrame(
        {
            "count_samples": bin_vals,
            "start_bin_value": bin_ranges[:-1],
            "end_bin_value": bin_ranges[1:],
        }
    )

    if method == 'median':
        bins_df["suspected_low_in_data_bins"] = bins_df["count_samples"] < (bins_df["count_samples"].median())
    else:
        bins_df["suspected_low_in_data_bins"] = bins_df["count_samples"] < (bins_df["count_samples"].mean() * threshold)

    return bins_df, X

### Asign bins

In [31]:
def assign_bins(
    X,
    y,
    samples_per_bin=20,
    max_bins=1000,
    method="mean",
    threshold=0.7,
):

    """
    Function which divides the data into bins
    Args:
        X, y: the data.
        samples_per_bin:
        max_bins:
        method:
        threshold:
    Return:
       bins_df:
       X:
    """
    num_of_bins = X.shape[0] // samples_per_bin
    if num_of_bins > max_bins:
        num_of_bins = max_bins
    bin_assignment, bin_ranges = pd.cut(y, num_of_bins, labels=False, retbins=True)
    # print('bin_assignment',bin_assignment)
    X["bins"] = bin_assignment
    # print('x',X)
    bin_vals = bin_assignment.value_counts().sort_index()
    # print('binvals',bin_vals)
    bin_vals = bin_vals.reindex(range(num_of_bins), fill_value=0)
    bin_ranges[0] = 0
    bin_ranges[-1] = bin_ranges[-1] * 10
    bins_df = pd.DataFrame(
        {
            "count_samples": bin_vals,
            "start_bin_value": bin_ranges[:-1],
            "end_bin_value": bin_ranges[1:],
        }
    )

    if method == 'median':
        bins_df["suspected_low_in_data_bins"] = bins_df["count_samples"] < (bins_df["count_samples"].median())
    else:
        bins_df["suspected_low_in_data_bins"] = bins_df["count_samples"] < (bins_df["count_samples"].mean() * threshold)
        print('mean threshold', bins_df["count_samples"].mean() * threshold)

    return bins_df, X

## Get confidence interval

In [32]:
def get_conf_interval(
    desired_conf_level,
    splited_data,
    seed
):

    """
    Function gets desired train confidence and calibration sets and a confidence level.
    It returns the confidence interval of the prediction as an array
    Args:
        desired_conf_level:
        splited_data: A dictionary of the data after splited
    Return:
       conf_preds:
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    rf_reg = RandomForestRegressor(
        random_state=1, n_estimators=200
    )  
    
    # Train a point prediction model
    rf_reg.fit(splited_data['X_pp_train'].values, splited_data['y_pp_train'].values)

    rf_preds = rf_reg.predict(splited_data['X_conf_train'].values)
    test_error = abs(rf_preds - splited_data['y_conf_train'].values)
    np.random.seed(seed)
    torch.manual_seed(seed)
    search_args = reg_mod.HnswGraphArgs(query_kwargs={"ef": 1500})
    bnds = reg_mod.SearchBounds(k_bounds=(5, 30))
    # bnds = reg_mod.SearchBounds(k_bounds=(1, 10))

    # Initialise and fit MACEST
    np.random.seed(seed)
    torch.manual_seed(seed)
    macest_model = reg_mod.ModelWithPredictionInterval(
        rf_reg,
        splited_data['X_conf_train'].values,
        test_error,
        error_dist="laplace",
        dist_func="linear",
        search_method_args=search_args,
    )

    optimiser_args = dict(popsize=20, disp=False)
    np.random.seed(seed)
    torch.manual_seed(seed)
    macest_model.fit(splited_data['X_cal'].values, splited_data['y_cal'].values, param_range=bnds, optimiser_args=optimiser_args)
    # predict the intervals based on the desired_conf_level
    np.random.seed(seed)
    torch.manual_seed(seed)
    conf_preds = macest_model.predict_interval(splited_data['X_test'].values, conf_level=desired_conf_level)

    return conf_preds


In [33]:
# # CQR
# def get_conf_interval(splited_data, desired_conf_level,seed=1):
#     # 1. Fit quantile regression model on training data
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     model = RandomForestQuantileRegressor().fit(splited_data['X_train'], splited_data['y_train'])
#     # 2. Make prediction on calibration data
#     y_cal_interval_pred = np.column_stack([
#         model.predict(splited_data['X_cal'], quantile=(desired_conf_level/2)*100), 
#         model.predict(splited_data['X_cal'], quantile=(1-desired_conf_level/2)*100)])
#     # 3. Compute conformity scores on calibration data
#     y_cal_conformity_scores = np.maximum(
#         y_cal_interval_pred[:,0] - splited_data['y_cal'], 
#         splited_data['y_cal'] - y_cal_interval_pred[:,1])
#     # 4. Get 1-alpha quantile from the distribution of conformity scores
#     #    Note: this is a single number
#     quantile_conformity_scores = np.quantile(
#         y_cal_conformity_scores, 1-desired_conf_level)
#     # 5. Make prediction on test data
#     y_test_interval_pred = np.column_stack([
#         model.predict(splited_data['X_test'], quantile=(desired_conf_level/2)*100), 
#         model.predict(splited_data['X_test'], quantile=(1-desired_conf_level/2)*100)])
#     # 6. Compute left (right) end of the interval by
#     #    subtracting (adding) the quantile to the predictions
#     y_test_interval_pred_cqr = np.column_stack([
#         y_test_interval_pred[:,0] - quantile_conformity_scores,
#         y_test_interval_pred[:,1] + quantile_conformity_scores])
    
#     return y_test_interval_pred_cqr


## Split the data into 4 groups

In [34]:
def split_bin(X, y):

  """
  Function....
  Args:
      X, y: the data

  Return:
      splited_data:
  """

  X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.15, random_state=10)
  X_pp_train, X_conf_train, y_pp_train, y_conf_train = train_test_split(X_train, y_train, test_size=0.333333333333, random_state=1)
  X_conf_train, X_cal, y_conf_train, y_cal, =  train_test_split(X_conf_train, y_conf_train, test_size=0.333333333333, random_state=1)

  splited_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'X_pp_train': X_pp_train,
    'y_pp_train': y_pp_train,
    'X_conf_train': X_conf_train,
    'X_cal': X_cal,
    'y_conf_train': y_conf_train,
    'y_cal': y_cal,
  }

  return splited_data


In [35]:
# # split for CQR
# def split_bin(X, y):

#   """
#   Function....
#   Args:
#       X, y: the data

#   Return:
#       splited_data:
#   """

#   X_train_and_cal, X_test, y_train_and_cal, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#   X_train, X_cal, y_train, y_cal = train_test_split(X_train_and_cal, y_train_and_cal, test_size=0.2, random_state=1)

#   splited_data = {
#     'X_train': X_train,
#     'X_test': X_test,
#     'y_train': y_train,
#     'y_test': y_test,
#     'X_cal': X_cal,
#     'y_cal': y_cal
#   }

#   return splited_data


In [36]:
def mutation_imptutation(data_bin,imputers,frac = 0.05,seed=1): #imputers should be list
  data_mut= data_bin.copy()
  np.random.seed(seed)
  idx = data_mut.sample(frac=0.05).index
  if len(idx)<1:
    np.random.seed(seed)
    idx = data_mut.sample(5).index
  # idx = np.random.randint(0, high=data_mut.shape[0], size=max(int(data_mut.shape[0]*0.05),5))
  data_mut.loc[idx] = imputers +[data_mut['bins'].iloc[0]]
  return data_mut

## Remove outlier from bins with Aleatoric uncertainty

In [37]:
def remove_outliers(df):
  """
  Removes outliers using pyOD package
  Args:
      df: Pandas DataFrame where X, y are stacked together

  Return:
      Pandas DataFrame without detected outliers
  """
  ODdata = df.copy()
  detector = KNN()
  detector.fit(ODdata)
  y_pred_od = detector.predict(df)
  ODdata = ODdata[y_pred_od == 0]
  return ODdata.reset_index(drop=True)

## Generate data for bin with epistemic uncertainty

In [38]:
def generate_data(data_bin,n_gen=1,seed=1):
  
  """
  Generating synthetic data per a given bin of data
  Args:
      data_bin: data to augment upon
      model_tvae: the data synthesis model generator
      n_gen: number of samples to generate where 1 equals the same size of original data 

  Return:
      generated new data
  """
  model_tvae = TVAE()
  np.random.seed(seed)
  torch.manual_seed(seed)
  model_tvae.fit(data_bin)

  np.random.seed(seed)
  torch.manual_seed(seed)
  new_data = model_tvae.sample(num_rows=len(data_bin)*n_gen)

  return new_data

# Pipeline Improvement Boston

Split data

In [55]:
bos_splited = split_bin(X_bos, y_bos)

print(f"X_train shape: {bos_splited['X_train'].shape}")
print(f"X_pp_train shape: {bos_splited['X_pp_train'].shape}")
print(f"X_conf_train shape: {bos_splited['X_conf_train'].shape}")
print(f"X_cal shape: {bos_splited['X_cal'].shape}")
print(f"X_test shape: {bos_splited['X_test'].shape}")

X_train shape: (430, 13)
X_pp_train shape: (286, 13)
X_conf_train shape: (96, 13)
X_cal shape: (48, 13)
X_test shape: (76, 13)


In [56]:
# bos_splited = split_bin(X_bos, y_bos)

# print(f"X_train shape: {bos_splited['X_train'].shape}")
# print(f"X_cal shape: {bos_splited['X_cal'].shape}")
# print(f"X_test shape: {bos_splited['X_test'].shape}")

 XGBOOST to initial predictions

In [57]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror')
xg_reg.fit(bos_splited['X_train'],bos_splited['y_train'])
preds = xg_reg.predict(bos_splited['X_test'])
rmse_org = np.sqrt(mean_squared_error(bos_splited['y_test'], preds))

print("RMSE: %f" % (rmse_org))

RMSE: 3.479232


## Apply Macest to get prediction interval

In [58]:
conf_interval = get_conf_interval(95, bos_splited,101)
# conf_interval = get_conf_interval(bos_splited, desired_conf_level = 0.05)

conf_df = pd.DataFrame()
conf_df['lower_bound_pred'] = conf_interval.squeeze()[:,0]
conf_df['upper_bound_pred'] = conf_interval.squeeze()[:,1]
conf_df['diff_prop'] = abs(conf_df['upper_bound_pred'] - conf_df['lower_bound_pred'])
conf_df['y_pred'] = preds

# Drop faulted samples:
conf_df = conf_df.query("lower_bound_pred<=y_pred <= upper_bound_pred").reset_index(drop=True)
display(conf_df)

,lower_bound_pred,upper_bound_pred,diff_prop,y_pred
0,16.668435,33.156565,16.488129,25.492981
1,20.042229,39.553771,19.511543,29.003119
2,19.546928,31.193072,11.646145,26.454632
3,20.673836,36.687164,16.013328,27.857496
4,15.155840,29.102160,13.946320,20.021534
...,...,...,...,...
70,17.902542,35.094458,17.191915,28.207253
71,13.216254,22.870746,9.654491,18.049124
72,24.218747,50.294253,26.075506,39.358990
73,18.899728,29.621272,10.721544,23.822235


## Bin the data

In [59]:
bins_df, _ = assign_bins(bos_splited['X_train'], bos_splited['y_train'])
bins_df

mean threshold 14.33333333333333


,count_samples,start_bin_value,end_bin_value,suspected_low_in_data_bins
0,6,0.000000,7.142857,True
1,13,7.142857,9.285714,True
2,15,9.285714,11.428571,False
3,28,11.428571,13.571429,False
4,37,13.571429,15.714286,False
5,32,15.714286,17.857143,False
6,59,17.857143,20.000000,False
7,63,20.000000,22.142857,False
8,62,22.142857,24.285714,False
9,28,24.285714,26.428571,False


## Analyze bin uncertainty

In [60]:
binslist = np.append(bins_df['start_bin_value'].values, bins_df['end_bin_value'].iloc[-1])
conf_df['bins']= pd.cut(conf_df['y_pred'] , bins = binslist,labels = bins_df.index.tolist())
conf_df

,lower_bound_pred,upper_bound_pred,diff_prop,y_pred,bins
0,16.668435,33.156565,16.488129,25.492981,9
1,20.042229,39.553771,19.511543,29.003119,11
2,19.546928,31.193072,11.646145,26.454632,10
3,20.673836,36.687164,16.013328,27.857496,10
4,15.155840,29.102160,13.946320,20.021534,7
...,...,...,...,...,...
70,17.902542,35.094458,17.191915,28.207253,10
71,13.216254,22.870746,9.654491,18.049124,6
72,24.218747,50.294253,26.075506,39.358990,16
73,18.899728,29.621272,10.721544,23.822235,8


In [61]:
diff_per_bin = pd.DataFrame(conf_df.groupby('bins')['diff_prop'].mean()).reset_index()

In [62]:
# threshold is the respected upper percentile that is suspected as unsertainty 
# The following bins are suspected as beeing with high unsertainty
percentile_threshold = 50

top_percentile = np.percentile(conf_df.groupby('bins')['diff_prop'].mean().dropna(), percentile_threshold)
uncertainty_bins = diff_per_bin[diff_per_bin.diff_prop > top_percentile]['bins']
uncertainty_bins

1      1
2      2
5      5
9      9
10    10
11    11
13    13
14    14
16    16
19    19
Name: bins, dtype: category
Categories (21, int64): [0 < 1 < 2 < 3 ... 17 < 18 < 19 < 20]

In [63]:
# Epistemic uncertainty
bins_df.loc[(bins_df.suspected_low_in_data_bins == True) & (bins_df.index.isin(uncertainty_bins) ) & (bins_df['count_samples'] > 1) ,'uncertainty_type'] = 'Epistemic'

# Another Epistemic uncertainty - if there are bins that didnt show up on the conf_df - suspect as epistemic
min_amount_samples_in_bin = 8
if len(diff_per_bin[diff_per_bin['diff_prop'].isna()].bins.tolist()) > 0:
  bins_df.loc[(bins_df.suspected_low_in_data_bins == True) & (bins_df['count_samples'] > min_amount_samples_in_bin) & (bins_df.index.isin(diff_per_bin[diff_per_bin['diff_prop'].isna()].bins.tolist()) ),'uncertainty_type'] = 'Epistemic'

# # Aleatoric uncertainty
bins_df.loc[(bins_df.suspected_low_in_data_bins == False) & (bins_df.index.isin(uncertainty_bins) ),'uncertainty_type'] = 'Aleatoric'

bins_df.sort_values(by = "count_samples",ascending=False).head(50)

,count_samples,start_bin_value,end_bin_value,suspected_low_in_data_bins,uncertainty_type
7,63,20.000000,22.142857,False,NaN
8,62,22.142857,24.285714,False,NaN
6,59,17.857143,20.000000,False,NaN
4,37,13.571429,15.714286,False,NaN
5,32,15.714286,17.857143,False,Aleatoric
3,28,11.428571,13.571429,False,NaN
9,28,24.285714,26.428571,False,Aleatoric
10,15,26.428571,28.571429,False,Aleatoric
11,15,28.571429,30.714286,False,Aleatoric
2,15,9.285714,11.428571,False,Aleatoric


### Remove outliers from each bin separately and then re-stack all data together

#### On full dataset

In [28]:
X_bos.shape

(506, 13)

In [ ]:
_,__=assign_bins(X_bos,y_bos)
bosdata = pd.concat([X_bos,y_bos], axis=1).reset_index(drop=True)
bostdata_no_alea = bosdata[bosdata['bins'].isin(bins_df[bins_df['uncertainty_type'] != 'Aleatoric'].index)].reset_index(drop=True)
data_no_ol = pd.concat([remove_outliers(bosdata[bosdata['bins'] == val]) for val in bins_df[bins_df['uncertainty_type'] == 'Aleatoric'].index])
bosdata_no_ol = pd.concat([bostdata_no_alea,data_no_ol]).reset_index(drop=True)

mean threshold 14.167999999999997


In [ ]:
bosdata_no_ol.shape

(463, 15)

#### On train only

In [64]:
bos_splited['X_train'].shape

(430, 14)

In [65]:
bosdata = pd.concat([bos_splited['X_train'],bos_splited['y_train']], axis=1).reset_index(drop=True)
bostdata_no_alea = bosdata[bosdata['bins'].isin(bins_df[bins_df['uncertainty_type'] != 'Aleatoric'].index)].reset_index(drop=True)
data_no_ol = pd.concat([remove_outliers(bosdata[bosdata['bins'] == val]) for val in bins_df[bins_df['uncertainty_type'] == 'Aleatoric'].index])
bosdata_no_ol = pd.concat([bostdata_no_alea,data_no_ol]).reset_index(drop=True)

In [66]:
bosdata_no_ol.shape

(423, 15)

## Generate data for uncertainties

In [67]:
print(bosdata_no_ol.shape)

(423, 15)


In [68]:
regenerated_data = pd.concat([generate_data(bosdata_no_ol[bosdata_no_ol['bins'] == val]) for val in bins_df[bins_df['uncertainty_type'] == 'Epistemic'].index])
bosdata_new = pd.concat([bosdata_no_ol,regenerated_data]).reset_index(drop=True)
print(bosdata_new.shape)

(458, 15)


In [69]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror')
xg_reg.fit(bosdata_new.drop(labels= ['MEDV','bins'],axis=1),bosdata_new.filter(['MEDV']))
preds = xg_reg.predict(bos_splited['X_test'])
rmse_imprv = np.sqrt(mean_squared_error(bos_splited['y_test'], preds))
print("\nOLD RMSE: %f" % (rmse_org))
print("\nImproved RMSE: %f" % (rmse_imprv))


OLD RMSE: 3.479232

Improved RMSE: 3.408730


In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(bosdata_new.iloc[:,:-1],bosdata_new.iloc[:,-1:], test_size=0.15, random_state=10)
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror')
xg_reg.fit(X_train,y_train)
preds_imprv = xg_reg.predict(X_test)
rmse_imprv = np.sqrt(mean_squared_error(y_test, preds_imprv))
print("\nOLD RMSE: %f" % (rmse_org))
print("\nImproved RMSE: %f" % (rmse_imprv))


OLD RMSE: 3.479232

Improved RMSE: 0.552427


# Pipeline Improvement Motors


Split data

In [ ]:
mot_splited = split_bin(X_mot, y_mot)

# print(f"X_train shape: {mot_splited['X_train'].shape}")
# print(f"X_pp_train shape: {mot_splited['X_pp_train'].shape}")
# print(f"X_conf_train shape: {mot_splited['X_conf_train'].shape}")
# print(f"X_cal shape: {mot_splited['X_cal'].shape}")
# print(f"X_test shape: {mot_splited['X_test'].shape}")

 XGBOOST to initial predictions

In [ ]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror')
xg_reg.fit(mot_splited['X_train'],mot_splited['y_train'])
preds = xg_reg.predict(mot_splited['X_test'])
rmse_org = np.sqrt(mean_squared_error(mot_splited['y_test'], preds))

print("RMSE: %f" % (rmse_org))

RMSE: 4.474360


## Apply Macest to get prediction interval

In [ ]:
# conf_interval = get_conf_interval(95, mot_splited,101)
conf_interval = get_conf_interval(mot_splited, desired_conf_level = 0.05) # cqr
conf_df = pd.DataFrame()
conf_df['lower_bound_pred'] = conf_interval.squeeze()[:,0]
conf_df['upper_bound_pred'] = conf_interval.squeeze()[:,1]
conf_df['diff_prop'] = abs(conf_df['upper_bound_pred'] - conf_df['lower_bound_pred'])
conf_df['y_pred'] = preds

# Drop faulted samples:
conf_df = conf_df.query("lower_bound_pred<=y_pred <= upper_bound_pred").reset_index(drop=True)
display(conf_df)

## Bin the data

In [ ]:
bins_df, _ = assign_bins(mot_splited['X_train'], mot_splited['y_train'])
# bins_df[bins_df['count_samples'] > 0]
bins_df

mean threshold 403.4177


,count_samples,start_bin_value,end_bin_value,suspected_low_in_data_bins
0,547471,0.000,0.732,False
1,14099,0.732,1.464,False
2,4881,1.464,2.196,False
3,2162,2.196,2.928,False
4,1323,2.928,3.660,False
...,...,...,...,...
995,0,728.340,729.072,True
996,0,729.072,729.804,True
997,0,729.804,730.536,True
998,0,730.536,731.268,True


## Analyze bin uncertainty

In [ ]:
binslist = np.append(bins_df['start_bin_value'].values, bins_df['end_bin_value'].iloc[-1])
conf_df['bins'] = pd.cut(conf_df['y_pred'], bins = binslist , labels = bins_df.index.tolist())
conf_df

,lower_bound_pred,upper_bound_pred,diff_prop,y_pred,bins
0,-1.376160,1.955901,3.332060,0.206000,0
1,-0.703299,0.762130,1.465429,0.115614,0
2,-0.819108,1.023993,1.843101,0.148109,0
3,-2.337047,2.744869,5.081917,0.232169,0
4,-1.337847,1.561112,2.898959,0.128422,0
...,...,...,...,...,...
97347,-2.170364,2.170364,4.340728,0.575086,0
97348,-0.829353,0.829353,1.658705,0.060426,0
97349,-0.406717,1.215929,1.622647,0.118797,0
97350,-1.085003,1.249393,2.334396,0.134827,0


In [ ]:
diff_per_bin = pd.DataFrame(conf_df.groupby('bins')['diff_prop'].mean()).reset_index()

In [ ]:
# threshold is the respected upper percentile that is suspected as unsertainty 
# The following bins are suspected as beeing with high unsertainty
percentile_threshold = 50

top_percentile = np.percentile(conf_df.groupby('bins')['diff_prop'].mean().dropna(), percentile_threshold)
uncertainty_bins = diff_per_bin[diff_per_bin.diff_prop > top_percentile]['bins']
uncertainty_bins

4    4
6    6
7    7
Name: bins, dtype: category
Categories (1000, int64): [0 < 1 < 2 < 3 ... 996 < 997 < 998 < 999]

In [ ]:
# # threshold is the respected upper percentile that is suspected as unsertainty 
# # The following bins are suspected as being with high uncertainty
# percentile_threshold = 10
# top_percentile = np.percentile(conf_df.groupby('bins')['diff_prop'].mean().dropna(), percentile_threshold)
# print('precentile',top_percentile)
# uncertainty_bins = diff_per_bin[diff_per_bin.diff_prop > top_percentile]
# uncertainty_bins

In [ ]:
# Epistemic uncertainty
bins_df.loc[(bins_df.suspected_low_in_data_bins == True) & (bins_df.index.isin(uncertainty_bins) ) & (bins_df['count_samples'] > 1) ,'uncertainty_type'] = 'Epistemic'

# Another Epistemic uncertainty - if there are bins that didnt show up on the conf_df - suspect as epistemic
min_amount_samples_in_bin = 8
if len(diff_per_bin[diff_per_bin['diff_prop'].isna()].bins.tolist()) > 0:
  bins_df.loc[(bins_df.suspected_low_in_data_bins == True) & (bins_df['count_samples'] > min_amount_samples_in_bin) & (bins_df.index.isin(diff_per_bin[diff_per_bin['diff_prop'].isna()].bins.tolist()) ),'uncertainty_type'] = 'Epistemic'

# # Aleatoric uncertainty
bins_df.loc[(bins_df.suspected_low_in_data_bins == False) & (bins_df.index.isin(uncertainty_bins) ),'uncertainty_type'] = 'Aleatoric'

bins_df.sort_values(by = "count_samples",ascending=False).head(50)

,count_samples,start_bin_value,end_bin_value,suspected_low_in_data_bins,uncertainty_type
0,547471,0.000,0.732,False,NaN
1,14099,0.732,1.464,False,NaN
2,4881,1.464,2.196,False,NaN
3,2162,2.196,2.928,False,NaN
4,1323,2.928,3.660,False,Aleatoric
5,1148,3.660,4.392,False,NaN
6,602,4.392,5.124,False,Aleatoric
17,443,12.444,13.176,False,NaN
7,408,5.124,5.856,False,Aleatoric
8,379,5.856,6.588,True,Epistemic


### Remove outliers from each bin separately and then re-stack all data together

#### On full dataset

In [ ]:
X_mot.shape

(67801, 10)

In [ ]:
_,__=assign_bins(X_mot,y_mot)
motdata = pd.concat([X_mot,y_mot], axis=1).reset_index(drop=True)
mot_data_no_alea = motdata[motdata['bins'].isin(bins_df[bins_df['uncertainty_type'] != 'Aleatoric'].index)].reset_index(drop=True)
data_no_ol = pd.concat([remove_outliers(motdata[motdata['bins'] == val]) for val in bins_df[bins_df['uncertainty_type'] == 'Aleatoric'].index])
motdata_no_ol = pd.concat([mot_data_no_alea,data_no_ol]).reset_index(drop=True)

mean threshold 47.460699999999996


In [ ]:
motdata_no_ol.shape

(67741, 11)

#### On train only

In [ ]:
mot_splited['X_train'].shape

(576311, 10)

In [ ]:
motdata = pd.concat([mot_splited['X_train'],mot_splited['y_train']], axis=1).reset_index(drop=True)
mot_data_no_alea = motdata[motdata['bins'].isin(bins_df[bins_df['uncertainty_type'] != 'Aleatoric'].index)].reset_index(drop=True)
data_no_ol = pd.concat([remove_outliers(motdata[motdata['bins'] == val]) for val in bins_df[bins_df['uncertainty_type'] == 'Aleatoric'].index])
motdata_no_ol = pd.concat([mot_data_no_alea,data_no_ol]).reset_index(drop=True)

In [ ]:
motdata_no_ol.shape

(576124, 11)

## Generate data for uncertainties

In [ ]:
print(motdata_no_ol.shape)

(576124, 11)


In [ ]:
regenerated_data = pd.concat([generate_data(motdata_no_ol[motdata_no_ol['bins'] == val]) for val in bins_df[bins_df['uncertainty_type'] == 'Epistemic'].index])
motdata_new = pd.concat([motdata_no_ol,regenerated_data]).reset_index(drop=True)
print(motdata_new.shape)

(579853, 11)


In [ ]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror')
xg_reg.fit(motdata_new.drop(labels= ['Frequency','bins'],axis=1),motdata_new.filter(['Frequency']))
preds = xg_reg.predict(mot_splited['X_test'])
rmse_imprv = np.sqrt(mean_squared_error(mot_splited['y_test'], preds))
print("\nOLD RMSE: %f" % (rmse_org))
print("\nImproved RMSE: %f" % (rmse_imprv))


OLD RMSE: 5.587417

Improved RMSE: 5.597457


In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(motdata_new.iloc[:,:-1],motdata_new.iloc[:,-1:], test_size=0.15, random_state=10)
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror')
xg_reg.fit(X_train,y_train)
preds_imprv = xg_reg.predict(X_test)
rmse_imprv = np.sqrt(mean_squared_error(y_test, preds_imprv))
print("\nOLD RMSE: %f" % (rmse_org))
print("\nImproved RMSE: %f" % (rmse_imprv))


OLD RMSE: 3.859696

Improved RMSE: 0.843254
